# Project 5: Systems of Linear Equations and College Football Team Ranking

In [1]:
import numpy as np
import scipy

In [2]:
mat_data = scipy.io.loadmat("scores.mat")

# Extract the Scores array
if "Scores" in mat_data:
    scores = mat_data["Scores"]
    print("Scores array shape:", scores.shape)
    print("Scores array:\n", scores)
else:
    raise ValueError("The 'Scores' variable was not found in the .mat file.")

Scores array shape: (10, 10)
Scores array:
 [[ 0  1  1 -1 -1  1 -1 -1 -1 -1]
 [-1  0  1 -1 -1 -1 -1 -1  1 -1]
 [-1 -1  0 -1 -1 -1 -1  1 -1 -1]
 [ 1  1  1  0 -1 -1  1  1  1 -1]
 [ 1  1  1  1  0  1  1  1  1  1]
 [-1  1  1  1 -1  0  1  1  1  1]
 [ 1  1  1 -1 -1 -1  0  1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1  0  1 -1]
 [ 1 -1  1 -1 -1 -1  1 -1  0 -1]
 [ 1  1  1  1 -1 -1  1  1 -1  0]]


In [3]:
# Define the system of equations
n = scores.shape[0]  # Number of teams
A = np.zeros((n, n))  # Coefficient matrix
b = np.zeros(n)  # Right-hand side vector

# Populate A and b based on the Scores array
for i in range(n):
    for j in range(n):
        if i != j:
            if scores[i, j] == 1:  # Team i beat Team j
                A[i, i] += 1
                A[i, j] -= 1
                b[i] += 1  # Team i gains a point
            elif scores[i, j] == -1:  # Team j beat Team i
                A[i, i] += 1
                A[i, j] -= 1
                b[i] -= 1  # Team i loses a point

# Add a constraint to ensure a unique solution (e.g., sum of rankings = 0)
A = np.vstack([A, np.ones(n)])  # Add a row for the constraint
b = np.append(b, 0)  # Add the corresponding value to b

In [4]:
# Gaussian Elimination
def gaussian_elimination(A, b):
    n = A.shape[0]  # Number of rows in the augmented matrix
    m = A.shape[1]  # Number of columns in the augmented matrix
    A = np.copy(A)  # Avoid modifying the original matrix
    b = np.copy(b)  # Avoid modifying the original vector

    for i in range(m):  # Iterate over columns (variables)
        # Partial pivoting
        max_row = max(range(i, n), key=lambda r: abs(A[r, i]))
        if abs(A[max_row, i]) < 1e-10:  # Check if the pivot is close to zero
            raise ValueError(
                "Matrix is singular or nearly singular. No unique solution exists."
            )

        # Swap rows if necessary
        if i != max_row:
            A[[i, max_row]] = A[[max_row, i]]  # Swap rows in A
            b[[i, max_row]] = b[[max_row, i]]  # Swap elements in b

        # Forward elimination
        for j in range(i + 1, n):
            factor = A[j, i] / A[i, i]
            A[j] -= factor * A[i]
            b[j] -= factor * b[i]

    # Back substitution
    x = np.zeros(m)
    for i in range(m - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1 :], x[i + 1 :])) / A[i, i]

    return x

In [5]:
# Solve for rankings
try:
    rankings = gaussian_elimination(A.astype(float), b.astype(float))
except ValueError as e:
    print(e)
    exit()

# Normalize rankings to ensure positive values
rankings = rankings - np.min(rankings) + 1

In [6]:
# Team names (example placeholder names)
teams = [
    "Team A",
    "Team B",
    "Team C",
    "Team D",
    "Team E",
    "Team F",
    "Team G",
    "Team H",
    "Team I",
    "Team J",
]

# Sort teams based on rankings
sorted_teams = sorted(zip(teams, rankings), key=lambda x: x[1], reverse=True)

# Display rankings
print("College Football Team Rankings:")
for rank, (team, score) in enumerate(sorted_teams, start=1):
    print(f"{rank}. {team}")

College Football Team Rankings:
1. Team E
2. Team F
3. Team D
4. Team J
5. Team G
6. Team I
7. Team H
8. Team A
9. Team B
10. Team C
